In [14]:
import sys
sys.path.append('/workspace/Documents')
# imports
import os, sys
import numpy as np 
import pandas as pd
import nibabel as nb
from skimage.measure import block_reduce 

import Diffusion_denoising_thin_slice.functions_collection as ff
import Diffusion_denoising_thin_slice.Data_processing as Data_processing

data_path = '/mnt/camca_NAS/Portable_CT_data'
save_path = '/mnt/camca_NAS/denoising'

In [6]:
# patient list
patient_list = pd.read_excel(os.path.join('/mnt/camca_NAS/diffusion_ct_motion/data/Patient_list','Patient_list_real_portable_CT_202404.xlsx'),dtype={'PatientID': str, 'AccessionNumber': str})

### check the CNR ratio

In [22]:
results = []
for i in range(0,len(patient_list)):
    patient_ID = patient_list['PatientID'][i]
    patient_subID = patient_list['AccessionNumber'][i]

    # original_data
    original_data = nb.load(os.path.join(data_path, 'nii_imgs_202404/motion',patient_ID,patient_subID,'portable/img.nii.gz'))
    # pixel dim
    pixdim = original_data.header.get_zooms()
    # print('pixel dim:', pixdim)

    # convert to xy 1mm first
    scale_factor = [int(1/pixdim[0]), int(1/pixdim[1]), 1]
    # use block_reduce to downsample the image
    original_data = original_data.get_fdata()
    if len(original_data.shape) == 4:
        original_data = original_data[:,:,:,0]
    img_1mm = block_reduce(original_data, tuple(scale_factor), np.mean)
    # print(img_1mm.shape)

    # put threshold [-100,100]
    img_1mm_cutoff = Data_processing.cutoff_intensity(img_1mm, cutoff_low=-100, cutoff_high=100)
    print('min:', np.min(img_1mm_cutoff), 'max:', np.max(img_1mm_cutoff))
    size = img_1mm_cutoff.shape

    # calculate CNR
    a = img_1mm_cutoff[size[0]//2-50:size[0]//2+50,size[1]//2-50:size[1]//2+50,size[2]//2 -50 :   size[2]//2+50]
    CNR = 100/np.std(a[ (a> 0) & (a < 100)])

    results.append([patient_ID, patient_subID, CNR])
    print('patient_ID:', patient_ID, 'patient_subID:', patient_subID, 'CNR:', CNR)

results = pd.DataFrame(results, columns=['PatientID', 'AccessionNumber', 'CNR'])
results.to_excel(os.path.join(save_path, 'Patient_lists/portable_CT_CNR.xlsx'), index=False)
   